# Introduction
This notebook demonstrates a simple bit flipping attack on data encrypted using AES in CBC mode. I've done this mostly to demonstrated it to myself, but it is great if you find it conceptually helpful as well.

This presentation is mostly about working through (with code) what I learned from reading [The dangers of AES-CBC](https://alicegg.tech/2019/06/23/aes-cbc.html). I found the explanation there to be clear and recommend checking it out.

# What is AES?
AES is is the [Advanced Encryption Standard](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard), also known as Rijndael. AES is a popular encryption algorithm. AES:
- Is a symmetric encryption algorithm (i.e., the same key is used to encrypt and decrypt the data)
- Is a block cipher (i.e., it encrypts fixed-size blocks of data; in the case of AES, blocks are 128 bits)
- Is specified to have three possible key sizes: 128, 196, and 256 bits

# What are Modes?
Block ciphers operate on a single block of data (again: 128 bits for AES). Modes specify how a block cipher is applied to data that is multiple blocks in size. The simplest mode is to just apply AES to each block. This approach, known as electronic codebook (ECB), is simple. However, it also has risky properties. Patterns in the plaintext are repeated throughout all of the ciphertext blocks, which allows for pattern analysis to attempt to glean information from the ciphertext. ECB is also susceptible to replay attacks since a given plaintext will always yield the same ciphertext in ECB.

Many modes exist, but this notebook is about a commonly-used mode called CBC.

## CBC Mode
Cipher block chaining (CBC) is a mode of operation that uses the ciphertext from the previously-encrypted block to XOR the plaintext of the current block. This effectively masks patterns that are present from block to block. During decryption the process is reversed by decrypting a block's ciphertext and then XORing the result with the previous block's ciphertext. This yields the original plaintext.

An edge case in the description above is that the first block has no previous block to use in the XOR process. For this first block a 128-bit initialization vector (IV) is randomly generated. The IV is stored and used during the decryption process to restore the first block's plaintext.

### Bit Flipping Attack
CBC is not sensitive to changes to the IV. That is, a different IV can be provided during decryption and the cipher won't notice. This will alter the plaintext resulting from decryption, of course, but this property can be exploited to alter the plaintext to result in something different. This can be effectively exploited if the attacker knows something about the structure of the plaintext. This notebook gives an example of a bit flipping to change the content of the first block in a way that is undetectable by the cipher.

#### Mitigation
If you still want to use CBC, a way to guard against tampering with the IV is to compute and store a message authentication code (MAC) using the IV and ciphertext. This approach is used, for example, in [Fernet](https://cryptography.io/en/latest/fernet/) in Python's [cryptography package](https://github.com/pyca/cryptography).

# AES-CBC Bit Flipping Example
The remainder of this notebook demonstrates a simple bit-flipping attack on an AES-CBC cipher.

## Setup and Encryption
We will be using the Python cryptography package. First, let's import dependencies.

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import secrets

Now, create the key, initialization vector, and cipher object.

In [2]:
CIPHER = algorithms.AES256
key = secrets.token_bytes(CIPHER.key_size // 8)
iv = secrets.token_bytes(CIPHER.block_size // 8)
cipher = Cipher(CIPHER(key), modes.CBC(iv))

Here's a simple plaintext message:

In [3]:
plaintext = b"a secret"

AES only operates on blocks of data that are exactly 128 bits in size. Blocks shorter than this need to be padded with extra data before being fed to AES. Let's do that using [PKCS7](https://cryptography.io/en/latest/hazmat/primitives/padding/#cryptography.hazmat.primitives.padding.PKCS7) and see what the plaintext looks like after padding.

In [4]:
padder = padding.PKCS7(CIPHER.block_size).padder()
padded_plaintext = padder.update(plaintext) + padder.finalize()

padded_plaintext

b'a secret\x08\x08\x08\x08\x08\x08\x08\x08'

Decryption is followed by an unpadding process, which we will see below.

We now have padded plaintext and an initialization vector. Those are the inputs needed to encrypt.

In [5]:
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

ciphertext

b'D\xac\x1f\xac_\x8e4\xff\xdb\xbe\n9\xfc\x17\xc8\xe0'

At this point, we have ciphertext containing our super-secret message ("a secret"). This ciphertext can be safely stored, along with the initialization vector.

## Unmodified Decryption
Given the ciphertext and initialization vector, we can restore the original plaintext.

In [6]:
decryptor = cipher.decryptor()
decrypted_ciphertext = decryptor.update(ciphertext) + decryptor.finalize()

decrypted_ciphertext

b'a secret\x08\x08\x08\x08\x08\x08\x08\x08'

That is the same as the padded plaintext from above! Next, we unpad the decrypted ciphertext to retrieve the original plaintext.

In [7]:
unpadder = padding.PKCS7(CIPHER.block_size).unpadder()
unpadded_decrypted_ciphertext = unpadder.update(decrypted_ciphertext) + unpadder.finalize()

unpadded_decrypted_ciphertext

b'a secret'

Great. We have retrieved the original data!

## Bit Flipping Attack
Now, let's assume that an attacker got ahold of the ciphertext and initialization vector. Additionally, this person knows something about the message and wants to change it. For example, let's say the attacker knows that the word "secret" is in the message and knows its location in the block. They don't have any practical way to figure out what else is in the message without the key. Nonetheless, the message can be altered to replace the word "secret" with anything else, like "rabbit".

The process to do this is simply determining the XOR of "secret" and "rabbit" and then XORing the IV by this result.

In [8]:
# First, compute the XOR between a block of text with "secret" and another with "beetle". This requires
# a little transformation of data to accomplish in Python.
secret_ord = [c for c in b"  secret        "]
rabbit_ord = [c for c in b"  rabbit        "]
secret_rabbit_xor = [s ^ r for s, r in zip(secret_ord, rabbit_ord)]

Now we need to XOR this result with the IV.

In [9]:
iv_ord = [c for c in iv]
altered_iv_arr = [i ^ sr for i, sr in zip(iv_ord, secret_rabbit_xor)]
altered_iv = bytes(altered_iv_arr)

# For comparison, here's the actual IV (see below).
display(iv)

# Here's the new IV.
altered_iv

b'\x86\xe0\x1eBE\x9d\x99\xc6\x00\xb9W\xa3\x7f5\xa7\xb3'

b'\x86\xe0\x1fFD\x8d\x95\xc6\x00\xb9W\xa3\x7f5\xa7\xb3'

Now let's do the decryption with this altered IV. This involves redoing a lot of the code from before because we need to initialize a new `Cipher` object using `altered_iv`.

In [10]:
cipher = Cipher(CIPHER(key), modes.CBC(altered_iv))
decryptor = cipher.decryptor()
decrypted_ciphertext = decryptor.update(ciphertext) + decryptor.finalize()
unpadder = padding.PKCS7(CIPHER.block_size).unpadder()
unpadded_decrypted_ciphertext = unpadder.update(decrypted_ciphertext) + unpadder.finalize()

unpadded_decrypted_ciphertext

b'a rabbit'

And there it is. Without knowledge of the key and limited knowledge of the message structure, we were able to alter the IV so that decryption yielded an altered message.